# Import Package

In [ ]:
#addface_4pose
#holdnosmallhand
#256frame
#gelu
#no_valid
#addmotionfeature
#addcon3dtf
#jicheng supplement model
!pip install -q tensorflow-addons==0.20.0
!pip install -q git+https://github.com/hoyso48/tf-utils@main
import gc
import json
import math
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import time
from tf_utils.learners import FGM, AWP
import warnings
warnings.filterwarnings("ignore")


In [ ]:
!pip install cached-property
from cached_property import cached_property
from shutil import copyfile

!pip install fastparquet
import fastparquet

!pip install Levenshtein
import Levenshtein as lev
starttime = time.time()

# SEED

In [ ]:
import random
SEED = 42
random.seed(SEED)       # Set seed for Python's random module
np.random.seed(SEED)    # Set seed for numpy
tf.random.set_seed(SEED) # Set seed for TensorFlow

# TPU setup for Colab TPU

In [ ]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  # strategy = tf.distribute.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  # print('REPLICA: ', strategy.num_replicas_in_sync)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
# strategy = tf.distribute.TPUStrategy(tpu)
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)
print('REPLICA: ', strategy.num_replicas_in_sync)

# TPU setup for Kaggle TPU

In [ ]:
# # https://www.kaggle.com/code/shlomoron/aslfr-ctc-on-tpu
# # Configure Strategy. Assume TPU...if not set default for GPU
# tpu = None
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("on TPU")
#     print("REPLICAS: ", strategy.num_replicas_in_sync)
# except:
#     strategy = tf.distribute.get_strategy()

In [ ]:
# https://www.kaggle.com/code/shlomoron/aslfr-ctc-on-tpu
##Uncommand this cell for Kaggle TPU
## copy our file into the working directory (make sure it has .py suffix)
# copyfile(src = "/kaggle/input/ctc-tpu/CTC_TPU.py", dst = "/kaggle/working//CTC_TPU.py")

# # import all our functions
# from CTC_TPU import classic_ctc_loss

# USE Supply or Val

In [ ]:
USE_VAL = True
USE_SUPPLY = False

# GCS_PATH

In [ ]:
GCS_PATH = {'ASL': 'gs://kds-a7bec0f3be4fe8073d283fc537263ae5223e9bbd2d812802cbcaf753',
            'new-coordinate-mean-std': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5',
            '5%-val-dataset': 'gs://kds-cfc32c8b51aa4053dc15c14ce0c260e474a660f82290940026a6c92b',
            '5%-train-dataset': 'gs://kds-309ee62af4d36117b93b96f5a8fcebdea475dcb64a7f895b077c2c95',
            '1%-val-dataset': 'gs://kds-5bded0c2ff71d0dd2f7d6306f2940737b2952ca4b6b7bf6864931eab',
            '1%-train-dataset': 'gs://kds-71da91630e0537bc8c375feb45e47971b0494b30ceb83d5990205c9b',
            'supply-pid': 'gs://kds-ca690717e0cb72e6e077d6c1c91998e28e234c7ab6e33f58c4665a1b',
            'supply-mean-std': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116',

      }
# load supply set
TRAIN_SUPPLY = tf.io.gfile.glob(GCS_PATH['supply-pid']+'/*.tfrecord')

# load train and val set, you can chose 95%/5% or 99%/1% spilt set 
VAL_FILENAMES =  tf.io.gfile.glob(GCS_PATH['1%-val-dataset']+'/*.tfrecord')
TRAIN_FILENAMES = tf.io.gfile.glob(GCS_PATH['1%-train-dataset']+'/*.tfrecord')

print(len(TRAIN_FILENAMES),len(VAL_FILENAMES))
print(len(TRAIN_SUPPLY))


COMPETITION_PATH = GCS_PATH['ASL']
!gsutil cp {COMPETITION_PATH}/train.csv .
!gsutil cp {COMPETITION_PATH}/character_to_prediction_index.json .
#!gsutil ls gs://kds-cd89af783a43be44bd46c8bb80046db59bf92ffaf527609ca13d24b8

# Setup frame and coordinate

In [ ]:
# #For Kaggle TPU
# with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
#     char_to_num = json.load(f)
    
with open ("/content/character_to_prediction_index.json", "r") as f:
    char_to_num = json.load(f)

pad_token = '^'
pad_token_idx = 59

char_to_num[pad_token] = pad_token_idx

num_to_char = {j:i for i,j in char_to_num.items()}

#df = pd.read_csv('/kaggle/input/asl-fingerspelling/train.csv')
df = pd.read_csv('train.csv')


LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
# new face_id
face_id = [454,356,323,361,389,288,251,264,447,366,368,
           401,397,435,284,301,372,345,383,367,365,352,433,
           376,298,265,93,234,300,132,340,353,127]
# LIP = list(set(LIP + face_id))
for k in face_id:
    if k not in LIP:
        LIP.append(k)
l = len(LIP)
print(len(LIP))
LIP  = LIP[:int(l-l/4)]
# LPOSE = [13, 15, 17, 19, 21]
# RPOSE = [14, 16, 18, 20, 22]
LPOSE = [1,3,5,7,9,11,13, 15,17, 19, 21,23,25,27,29,31]#
RPOSE = [0,2,4,6,8,10,12, 14,16, 18, 20,22,24,26,28,30]#
POSE = LPOSE + RPOSE

X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE] + [f'x_face_{i}' for i in LIP]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE] + [f'y_face_{i}' for i in LIP]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE] + [f'z_face_{i}' for i in LIP]

SEL_COLS = X + Y + Z
FRAME_LEN = 256#128
MAX_PHRASE_LENGTH = 64

LIP_IDX_X   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "x" in col]
RHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "x" in col]
LHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "x" in col]
RPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col.split('_')[-1]) in RPOSE and "x" in col]
LPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col.split('_')[-1]) in LPOSE and "x" in col]

LIP_IDX_Y   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "y" in col]
RHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "y" in col]
LHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "y" in col]
RPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col.split('_')[-1]) in RPOSE and "y" in col]
LPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col.split('_')[-1]) in LPOSE and "y" in col]

LIP_IDX_Z   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "z" in col]
RHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "z" in col]
LHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "z" in col]
RPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col.split('_')[-1]) in RPOSE and "z" in col]
LPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col.split('_')[-1]) in LPOSE and "z" in col]

# check path for supply-mean-std
if USE_SUPPLY:
  paths = {
      'RHM': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/rh_mean.npy',
      'LHM': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/lh_mean.npy',
      'RPM': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/rp_mean.npy',
      'LPM': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/lp_mean.npy',
      'LIPM': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/lip_mean.npy',
      'RHS': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/rh_std.npy',
      'LHS': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/lh_std.npy',
      'RPS': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/rp_std.npy',
      'LPS': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/lp_std.npy',
      'LIPS': 'gs://kds-ab9973aa093552472b53710e8b9bcaac6ed07fe79ca397062e266116/lip_std.npy'
  }

  loaded_data = {key: np.load(tf.io.gfile.GFile(path, 'rb')) for key, path in paths.items()}

  RHM = loaded_data['RHM']
  LHM = loaded_data['LHM']
  RPM = loaded_data['RPM']
  LPM = loaded_data['LPM']
  LIPM = loaded_data['LIPM']

  RHS = loaded_data['RHS']
  LHS = loaded_data['LHS']
  RPS = loaded_data['RPS']
  LPS = loaded_data['LPS']
  LIPS = loaded_data['LIPS']

# check path for new-coordinate-mean-std   
if USE_VAL:
  paths = {
      'RHM': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/rh_mean.npy',
      'LHM': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/lh_mean.npy',
      'RPM': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/rp_mean.npy',
      'LPM': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/lp_mean.npy',
      'LIPM': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/lip_mean.npy',
      'RHS': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/rh_std.npy',
      'LHS': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/lh_std.npy',
      'RPS': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/rp_std.npy',
      'LPS': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/lp_std.npy',
      'LIPS': 'gs://kds-eccfa04ab4e08b5b48389f4abb31fb092c6cfd0394610d8929a376b5/lip_std.npy'
  }

  loaded_data = {key: np.load(tf.io.gfile.GFile(path, 'rb')) for key, path in paths.items()}

  RHM = loaded_data['RHM']
  LHM = loaded_data['LHM']
  RPM = loaded_data['RPM']
  LPM = loaded_data['LPM']
  LIPM = loaded_data['LIPM']

  RHS = loaded_data['RHS']
  LHS = loaded_data['LHS']
  RPS = loaded_data['RPS']
  LPS = loaded_data['LPS']
  LIPS = loaded_data['LIPS']

# Proprocessing

In [ ]:

@tf.function()
def resize_pad(x):
    if tf.shape(x)[0] < FRAME_LEN:
        x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0], [0, 0]]), constant_values=float("NaN"))
    else:
        x = tf.image.resize(x, (FRAME_LEN, tf.shape(x)[1]))
    return x

@tf.function(jit_compile=True)
def pre_process0(x):
    lip_x = tf.gather(x, LIP_IDX_X, axis=1)
    lip_y = tf.gather(x, LIP_IDX_Y, axis=1)
    lip_z = tf.gather(x, LIP_IDX_Z, axis=1)

    rhand_x = tf.gather(x, RHAND_IDX_X, axis=1)
    rhand_y = tf.gather(x, RHAND_IDX_Y, axis=1)
    rhand_z = tf.gather(x, RHAND_IDX_Z, axis=1)

    lhand_x = tf.gather(x, LHAND_IDX_X, axis=1)
    lhand_y = tf.gather(x, LHAND_IDX_Y, axis=1)
    lhand_z = tf.gather(x, LHAND_IDX_Z, axis=1)

    rpose_x = tf.gather(x, RPOSE_IDX_X, axis=1)
    rpose_y = tf.gather(x, RPOSE_IDX_Y, axis=1)
    rpose_z = tf.gather(x, RPOSE_IDX_Z, axis=1)

    lpose_x = tf.gather(x, LPOSE_IDX_X, axis=1)
    lpose_y = tf.gather(x, LPOSE_IDX_Y, axis=1)
    lpose_z = tf.gather(x, LPOSE_IDX_Z, axis=1)

    lip   = tf.concat([lip_x[..., tf.newaxis], lip_y[..., tf.newaxis], lip_z[..., tf.newaxis]], axis=-1)
    rhand = tf.concat([rhand_x[..., tf.newaxis], rhand_y[..., tf.newaxis], rhand_z[..., tf.newaxis]], axis=-1)
    lhand = tf.concat([lhand_x[..., tf.newaxis], lhand_y[..., tf.newaxis], lhand_z[..., tf.newaxis]], axis=-1)
    rpose = tf.concat([rpose_x[..., tf.newaxis], rpose_y[..., tf.newaxis], rpose_z[..., tf.newaxis]], axis=-1)
    lpose = tf.concat([lpose_x[..., tf.newaxis], lpose_y[..., tf.newaxis], lpose_z[..., tf.newaxis]], axis=-1)

    hand = tf.concat([rhand, lhand,lip,rpose,lpose], axis=1)
    hand = tf.where(tf.math.is_nan(hand), 0.0, hand)
    mask = tf.math.not_equal(tf.reduce_sum(hand, axis=[1, 2]), 0.0)

    lip = lip[mask]
    rhand = rhand[mask]
    lhand = lhand[mask]
    rpose = rpose[mask]
    lpose = lpose[mask]

    return lip, rhand, lhand, rpose, lpose

@tf.function()
def pre_process1(lip, rhand, lhand, rpose, lpose):
    lip   = (resize_pad(lip) - LIPM) / LIPS
    rhand = (resize_pad(rhand) - RHM) / RHS
    lhand = (resize_pad(lhand) - LHM) / LHS
    rpose = (resize_pad(rpose) - RPM) / RPS
    lpose = (resize_pad(lpose) - LPM) / LPS

    x = tf.concat([lip, rhand, lhand, rpose, lpose], axis=1)
    ##motionfeature
    length = tf.shape(x)[0]
    dx = tf.cond(tf.shape(x)[0]>1,lambda:tf.pad(x[1:] - x[:-1], [[0,1],[0,0],[0,0]]),lambda:tf.zeros_like(x))
    dx2 = tf.cond(tf.shape(x)[0]>2,lambda:tf.pad(x[2:] - x[:-2], [[0,2],[0,0],[0,0]]),lambda:tf.zeros_like(x))
    x = tf.concat([x,dx,dx2], axis = -1)
    s = tf.shape(x)
    x = tf.reshape(x, (s[0], s[1]*s[2]))

    x = tf.where(tf.math.is_nan(x), 0.0, x)
    return x

INPUT_SHAPE = [256,1152]

# pre0 = pre_process0(frames)
# pre1 = pre_process1(*pre0)
# INPUT_SHAPE = list(pre1.shape)
# print(INPUT_SHAPE)
# pre1

In [ ]:
def decode_fn(record_bytes):
    schema = {
        "lip": tf.io.VarLenFeature(tf.float32),
        "rhand": tf.io.VarLenFeature(tf.float32),
        "lhand": tf.io.VarLenFeature(tf.float32),
        "rpose": tf.io.VarLenFeature(tf.float32),
        "lpose": tf.io.VarLenFeature(tf.float32),
        "phrase": tf.io.VarLenFeature(tf.int64)
    }
    x = tf.io.parse_single_example(record_bytes, schema)

    lip = tf.reshape(tf.sparse.to_dense(x["lip"]), (-1, 54, 3))
    rhand = tf.reshape(tf.sparse.to_dense(x["rhand"]), (-1, 21, 3))
    lhand = tf.reshape(tf.sparse.to_dense(x["lhand"]), (-1, 21, 3))
    rpose = tf.reshape(tf.sparse.to_dense(x["rpose"]), (-1, 16, 3))
    lpose = tf.reshape(tf.sparse.to_dense(x["lpose"]), (-1, 16, 3))
    phrase = tf.sparse.to_dense(x["phrase"])

    return lip, rhand, lhand, rpose, lpose, phrase

def pre_process_fn(lip, rhand, lhand, rpose, lpose, phrase):
    phrase = tf.pad(phrase, [[0, MAX_PHRASE_LENGTH-tf.shape(phrase)[0]]], constant_values=pad_token_idx)
    return pre_process1(lip, rhand, lhand, rpose, lpose), phrase



# Random drop frame Augmentation

In [ ]:
def random_zero_out_sequence(sequence):
    """
    Set the entire sequence to zero.
    """
    return tf.zeros_like(sequence)

def decode_fn_with_random_drop(lip, rhand, lhand, rpose, lpose, drop_prob=0.025):

    # Choose one of the sequences randomly based on the drop probability
    drop_choice = tf.cond(tf.random.uniform(()) < drop_prob,
                          lambda: tf.random.uniform((), 0, 5, dtype=tf.int32),
                          lambda: tf.constant(-1, dtype=tf.int32))

    lip = tf.cond(tf.equal(drop_choice, 0), lambda: random_zero_out_sequence(lip), lambda: lip)
    rhand = tf.cond(tf.equal(drop_choice, 1), lambda: random_zero_out_sequence(rhand), lambda: rhand)
    lhand = tf.cond(tf.equal(drop_choice, 2), lambda: random_zero_out_sequence(lhand), lambda: lhand)
    rpose = tf.cond(tf.equal(drop_choice, 3), lambda: random_zero_out_sequence(rpose), lambda: rpose)
    lpose = tf.cond(tf.equal(drop_choice, 4), lambda: random_zero_out_sequence(lpose), lambda: lpose)

    return lip, rhand, lhand, rpose, lpose

def spatial_random_affine(xyz,
    scale  = (0.9,1.1),
    shear = (-0.1,0.1),
    shift  = (-0.05,0.05),
    degree = (-10,10),
):
    center = tf.constant([0.5,0.5])
    if scale is not None:
        scale = tf.random.uniform((),*scale)
        xyz = scale*xyz

    if shear is not None:
        xy = xyz[...,:2]
        z = xyz[...,2:]
        shear_x = shear_y = tf.random.uniform((),*shear)
        if tf.random.uniform(()) < 0.5:
            shear_x = 0.
        else:
            shear_y = 0.
        shear_mat = tf.identity([
            [1.,shear_x],
            [shear_y,1.]
        ])
        xy = xy @ shear_mat
        center = center + [shear_y, shear_x]
        xyz = tf.concat([xy,z], axis=-1)

    if degree is not None:
        xy = xyz[...,:2]
        z = xyz[...,2:]
        xy -= center
        degree = tf.random.uniform((),*degree)
        radian = degree/180*np.pi
        c = tf.math.cos(radian)
        s = tf.math.sin(radian)
        rotate_mat = tf.identity([
            [c, s],
            [-s, c],
        ])
        xy = xy @ rotate_mat
        xy = xy + center
        xyz = tf.concat([xy,z], axis=-1)

    if shift is not None:
        shift = tf.random.uniform((),*shift)
        xyz = xyz + shift

    return xyz

def augmented_fn(lip, rhand, lhand, rpose, lpose, phrase):

    # Apply spatial random affine
    if tf.random.uniform(()) < 0.9:
        features = tf.concat([lip, rhand, lhand, rpose, lpose], axis=1)
        features = spatial_random_affine(features)
        lip, rhand, lhand, rpose, lpose = tf.split(features, axis=1, num_or_size_splits=[54, 21, 21, 16, 16])

    # Apply the random drop augmentation
    lip, rhand, lhand, rpose, lpose = decode_fn_with_random_drop(lip, rhand, lhand, rpose, lpose)

    return lip, rhand, lhand, rpose, lpose, phrase

# Train and Val_Set

In [ ]:
train_batch_size = 32
val_batch_size = 32

supply_tffiles = TRAIN_SUPPLY
val_tffiles = VAL_FILENAMES
train_tffiles = TRAIN_FILENAMES

random.seed(SEED)
random.shuffle(val_tffiles)
random.shuffle(train_tffiles)
random.shuffle(supply_tffiles)


# Trianing on supplyment dataset

if USE_SUPPLY:
  supply_dataset_pre = (tf.data.TFRecordDataset(supply_tffiles)
                            .prefetch(tf.data.AUTOTUNE)
                            .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)
                            .map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE))

# Use Validation set
if USE_VAL:
  val_dataset_pre =  (tf.data.TFRecordDataset(val_tffiles)
                          .prefetch(tf.data.AUTOTUNE)
                          .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)
                          .map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE))

  train_dataset_pre =  (tf.data.TFRecordDataset(train_tffiles)
                          .prefetch(tf.data.AUTOTUNE)
                          .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)
                          .map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE))



# Generate val and train dataset

In [ ]:
if USE_SUPPLY:
  train_supply_items = [x for x in supply_dataset_pre]
  train_supply_items_X = [x[0] for x in train_supply_items]
  train_supply_items_y = [tf.cast(x[1], dtype = tf.int32) for x in train_supply_items]
  train_supply_dataset = tf.data.Dataset.from_tensor_slices((train_supply_items_X, train_supply_items_y)).prefetch(tf.data.AUTOTUNE).shuffle(60000).batch(train_batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
  batch = next(iter(train_supply_dataset))
  print(len(train_supply_dataset))

if USE_VAL:
  val_items = [x for x in val_dataset_pre]
  val_items_X = [x[0] for x in val_items]
  val_items_y = [tf.cast(x[1], dtype = tf.int32) for x in val_items]
  val_dataset = tf.data.Dataset.from_tensor_slices((val_items_X, val_items_y)).prefetch(tf.data.AUTOTUNE).batch(val_batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

  train_items = [x for x in train_dataset_pre]
  train_items_X = [x[0] for x in train_items]
  train_items_y = [tf.cast(x[1], dtype = tf.int32) for x in train_items]
  train_dataset = tf.data.Dataset.from_tensor_slices((train_items_X, train_items_y)).prefetch(tf.data.AUTOTUNE).shuffle(60000).batch(train_batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
  batch = next(iter(val_dataset))
  print(batch[0].shape, batch[1].shape)
  print(len(train_dataset))
  print(len(val_dataset))
  nums_epoch_train = len(train_dataset)

# Model Config

In [ ]:
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self,
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True

    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        #保留残差网络
        # if (channels_in == channel_size):
        x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x


def TransformerBlock(dim=256, num_heads=6, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

def positional_encoding(maxlen, num_hid):
        depth = num_hid/2
        positions = tf.range(maxlen, dtype = tf.float32)[..., tf.newaxis]
        depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
        angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
        angle_rads = tf.linalg.matmul(positions, angle_rates)
        pos_encoding = tf.concat(
          [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
          axis=-1)
        return pos_encoding

In [ ]:
# Initiailizers
INIT_HE_UNIFORM = tf.keras.initializers.he_uniform
INIT_GLOROT_UNIFORM = tf.keras.initializers.glorot_uniform
INIT_ZEROS = tf.keras.initializers.constant(0.0)
# Activations
GELU = tf.keras.activations.gelu

LHAND_IDX = LHAND_IDX_X + LHAND_IDX_Y + LHAND_IDX_Z
RHAND_IDX = RHAND_IDX_X + RHAND_IDX_Y + RHAND_IDX_Z
LIP_IDX = LIP_IDX_X + LIP_IDX_Y + LIP_IDX_Z
POSE_IDX = LPOSE_IDX_X + RPOSE_IDX_X + LPOSE_IDX_Y + RPOSE_IDX_Y + LPOSE_IDX_Z + RPOSE_IDX_Z

# Landmark Embedding
# Embeds a landmark using local features and global features
class LandmarkEmbedding(tf.keras.Model):
    def __init__(self, units, name='landmark_embedding'):
        super(LandmarkEmbedding, self).__init__(name=name)
        self.units = units
        self.supports_masking = True

    def build(self, input_shape):
        self.empty_embedding = self.add_weight(
            name=f'{self.name}_empty_embedding',
            shape=[self.units],
            initializer=INIT_ZEROS,
        )
        # local feature extractor
        self.lefthand_mlp = tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_lefthand', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM, activation=GELU)
        self.righthand_mlp = tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_righthand', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM, activation=GELU)
        self.lips_mlp = tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_lips', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM, activation=GELU)
        self.pose_mlp = tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_pose', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM, activation=GELU)

        # full feature extractor
        self.full_mlp = tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_full', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM, activation=GELU)

        # global feature extractor
        self.global_mlp = tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_global', use_bias=False, kernel_initializer=INIT_HE_UNIFORM)

    def call(self, x):
        return tf.where(
                # Checks whether landmark is missing in frame
                tf.reduce_sum(x, axis=2, keepdims=True) == 0,
                # If so, the empty embedding is used
                self.empty_embedding,
                # Otherwise the landmark data is embedded
                self.global_mlp(tf.concat([self.full_extractor(x), self.local_extractor(x)], axis=-1))
            )

    def local_extractor(self, x):

        lefthand_feature = self.lefthand_mlp(tf.gather(x, LHAND_IDX, axis=-1))
        righthad_featrue = self.righthand_mlp(tf.gather(x, RHAND_IDX, axis=-1))
        # Keeps dominant hand feature
        hand_feature = tf.reduce_max(tf.stack([lefthand_feature, righthad_featrue], axis=-1), axis=-1)
        lips_featrue = self.lips_mlp(tf.gather(x, LIP_IDX, axis=-1))
        pose_featrue = self.pose_mlp(tf.gather(x, POSE_IDX, axis=-1))

        return tf.concat([hand_feature, lips_featrue, pose_featrue], axis=-1)

    def full_extractor(self, x):
        return self.full_mlp(x)

# CTCLOSS with label smoothing

In [ ]:
def CTCLoss(labels, logits):
    label_length = tf.reduce_sum(tf.cast(labels != pad_token_idx, tf.int32), axis=-1)
    logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    loss = tf.nn.ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=pad_token_idx,
            logits_time_major=False
        )
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
# https://www.kaggle.com/code/shlomoron/aslfr-ctc-on-tpu
# def CTCLoss(labels, logits):
#     label_length = tf.reduce_sum(tf.cast(labels != pad_token_idx, tf.int32), axis=-1)
#     logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    
#     loss = classic_ctc_loss(
#             labels=labels,
#             logits=logits,
#             label_length=label_length,
#             logit_length=logit_length,
#             blank_index=pad_token_idx,
#         )
#     '''
#     loss = tf.nn.ctc_loss(
#             labels=labels,
#             logits=logits,
#             label_length=label_length,
#             logit_length=logit_length,
#             blank_index=pad_token_idx,
#             logits_time_major=False
#         )
#     '''
#     loss = tf.reduce_mean(loss)
#     return loss

In [ ]:
def smooth_ctc_loss(labels, logits, num_classes = 60 , blank=0, weight=0.7):
    # Compute CTC Loss
    label_length = tf.reduce_sum(tf.cast(labels != pad_token_idx, tf.int32), axis=-1)
    logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    ctc_loss = tf.nn.ctc_loss(
        labels=labels,
        logits=logits,
        label_length=label_length,
        logit_length=logit_length,
        blank_index=pad_token_idx,
        logits_time_major=False
    )

    '''
    Use this for Kaggle TPU
    ctc_loss = classic_ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=pad_token_idx,
        )
    '''    
    ctc_loss = tf.reduce_mean(ctc_loss)

    # Compute KL Divergence Loss
    # Compute the logarithm of probabilities. Avoid taking the log of zero by adding a small constant.
    kl_inp = tf.nn.softmax(logits)

    # Create the target distribution
    kl_tar = tf.fill(tf.shape(logits), 1. / num_classes)

    # Compute the KL divergence
    kldiv_loss = (tf.keras.losses.KLDivergence(tf.keras.losses.Reduction.NONE)(kl_tar, kl_inp)
                 + tf.keras.losses.KLDivergence(tf.keras.losses.Reduction.NONE)(kl_inp, kl_tar))/2.0

    kldiv_loss = tf.reduce_mean(kldiv_loss)

    # Combined Loss
    loss = (1. - weight) * ctc_loss + weight * kldiv_loss
    #loss = ctc_loss
    #loss = kldiv_loss
    return loss

# Get Model

In [ ]:
def get_model(dim = 384, num_blocks = 6, drop_rate = 0.4):
    with strategy.scope():
      inp = tf.keras.Input(INPUT_SHAPE)
      x = tf.keras.layers.Masking(mask_value=0.0)(inp)
      # original
      x = tf.keras.layers.Dense(dim, use_bias=False, name='stem_conv')(x)
      # local & global extractor
      #x = LandmarkEmbedding(dim)(x)
      pe = tf.cast(positional_encoding(INPUT_SHAPE[0], dim), dtype=x.dtype)
      x = x + pe
      x = tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn')(x)

      for i in range(num_blocks):
          x = Conv1DBlock(dim, 11, drop_rate=drop_rate)(x)
          x = Conv1DBlock(dim, 5, drop_rate=drop_rate)(x)
          x = Conv1DBlock(dim, 3, drop_rate=drop_rate)(x)
          x = TransformerBlock(dim, expand=2)(x)

      x = tf.keras.layers.Dense(dim*2,tf.keras.activations.gelu,name='top_conv')(x)
      x = tf.keras.layers.Dropout(0.4)(x)
      x = tf.keras.layers.Dense(len(char_to_num),name='classifier')(x)

      model = tf.keras.Model(inp, x)

      loss = smooth_ctc_loss

      # Adam Optimizer
      optimizer = tfa.optimizers.RectifiedAdam(sma_threshold=4)
      optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=5)

      model.compile(loss=loss, optimizer=optimizer)

      return model

#steps_per_epoch = 61955//train_batch_size
tf.keras.backend.clear_session()
model = get_model()
model(batch[0])
model.summary()

In [ ]:
def num_to_char_fn(y):
    return [num_to_char.get(x, "") for x in y]

@tf.function()
def decode_phrase(pred):
    x = tf.argmax(pred, axis=1)
    diff = tf.not_equal(x[:-1], x[1:])
    adjacent_indices = tf.where(diff)[:, 0]
    x = tf.gather(x, adjacent_indices)
    mask = x != pad_token_idx
    x = tf.boolean_mask(x, mask, axis=0)
    return x

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    output_text = []
    for result in pred:
        result = "".join(num_to_char_fn(decode_phrase(result).numpy()))
        output_text.append(result)
    return output_text

# Epoch Learning Rate

In [ ]:
N_EPOCHS = 80 if USE_SUPPLY else 120
N_WARMUP_EPOCHS = 10
LR_MAX = 1e-3*0.5
WD_RATIO = 0.05
WARMUP_METHOD = "exp"

# Callback

In [ ]:
from Levenshtein import distance
if USE_SUPPLY:
  val_set = [x for x in train_supply_dataset.take(5)]
elif USE_VAL:
  val_set = [x for x in val_dataset]

#/kaggle/input/asl-fingerspelling/character_to_prediction_index.json
with open ("/content/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {j:i for i,j in character_map.items()}

class val_lev_callback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
    def on_epoch_end(self, epoch: int, logs=None):
        if epoch < N_EPOCHS-20:
          return
        calculate_val_lev()

def calculate_val_lev():
    preds = []
    targets = []
    scores = []
    for batch_idx in range(len(val_set)):
        preds_batch = model.predict(val_set[batch_idx][0], verbose = 0)
        targets_batch = val_set[batch_idx][1]
        for pred_idx in range(len(preds_batch)):
            preds.append("".join([rev_character_map.get(s, "") for s in decode_phrase(preds_batch[pred_idx]).numpy()]))
            targets.append("".join([rev_character_map.get(s, "") for s in targets_batch[pred_idx].numpy()]))

    N = [len(phrase) for phrase in targets]
    lev_dist = [distance(preds[i], targets[i]) for i in range(len(targets))]
    print('Lev distance: '+str((np.sum(N) - np.sum(lev_dist))/np.sum(N)))
    for _ in range(8):
      i = tf.random.uniform(shape=[], minval=0, maxval=len(targets), dtype=tf.int32).numpy()  # Corrected this line
      print(f'predic: {preds[i]}')
      print(f'target: {targets[i]}')
      print("-" * 100)

In [ ]:
# Custom callback to update weight decay with learning rate
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=WD_RATIO):
        self.step_counter = 0
        self.wd_ratio = wd_ratio

    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

class save_model_callback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
    def on_epoch_end(self, epoch: int, logs=None):
          if (N_EPOCHS-30 <= epoch < N_EPOCHS-10 and (epoch) % 5 == 0) or epoch >= N_EPOCHS-10 :
            self.model.save_weights(f"model_epoch_{epoch}.h5")

In [ ]:
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=N_EPOCHS):

    if current_step < num_warmup_steps:
        if WARMUP_METHOD == 'log':
            return lr_max * 0.10 ** (num_warmup_steps - current_step)
        else:
            return lr_max * 2 ** -(num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max

def plot_lr_schedule(lr_schedule, epochs):
    fig = plt.figure(figsize=(20, 10))
    plt.plot([None] + lr_schedule + [None])
    # X Labels
    x = np.arange(1, epochs + 1)
    x_axis_labels = [i if epochs <= 40 or i % 5 == 0 or i == 1 else None for i in range(1, epochs + 1)]
    plt.xlim([1, epochs])
    plt.xticks(x, x_axis_labels) # set tick step to 1 and let x axis start at 1

    # Increase y-limit for better readability
    plt.ylim([0, max(lr_schedule) * 1.1])

    # Title
    schedule_info = f'start: {lr_schedule[0]:.1E}, max: {max(lr_schedule):.1E}, final: {lr_schedule[-1]:.1E}'
    plt.title(f'Step Learning Rate Schedule, {schedule_info}', size=18, pad=12)

    # Plot Learning Rates
    for x, val in enumerate(lr_schedule):
        if epochs <= 40 or x % 5 == 0 or x is epochs - 1:
            if x < len(lr_schedule) - 1:
                if lr_schedule[x - 1] < val:
                    ha = 'right'
                else:
                    ha = 'left'
            elif x == 0:
                ha = 'right'
            else:
                ha = 'left'
            plt.plot(x + 1, val, 'o', color='black');
            offset_y = (max(lr_schedule) - min(lr_schedule)) * 0.02
            plt.annotate(f'{val:.1E}', xy=(x + 1, val + offset_y), size=12, ha=ha)

    plt.xlabel('Epoch', size=16, labelpad=5)
    plt.ylabel('Learning Rate', size=16, labelpad=5)
    plt.grid()
    plt.show()

# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=N_WARMUP_EPOCHS, lr_max=LR_MAX, num_cycles=0.50) for step in range(N_EPOCHS)]
# Plot Learning Rate Schedule
plot_lr_schedule(LR_SCHEDULE, epochs=N_EPOCHS)
# Learning Rate Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=0)

# Custom callback to update weight decay with learning rate
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=WD_RATIO):
        self.step_counter = 0
        self.wd_ratio = wd_ratio

    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Specify the path where you want to save the model checkpoint
checkpoint_path = "model_checkpoint.h5"

# Create the checkpoint callback, specifying details such as the metric to monitor, mode, etc.
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True, # Set to False if you want to save the entire model
    monitor='loss',         # Monitor validation loss. Change this if you want to monitor another metric.
    mode='min',             # Mode = 'min' ensures that the model is saved only when the monitored metric (validation loss in this case) decreases.
    save_best_only=True,    # Only save a model if `loss` has improved.
    verbose=0               # Log when a new best is saved
)



# Training on Supply

In [ ]:
if USE_SUPPLY:
  history = model.fit(
      train_supply_dataset,
      #validation_data=val_dataset,
      epochs=N_EPOCHS ,
      verbose = 2,
      callbacks=[
          save_model_callback(),
          lr_callback,
          WeightDecayCallback(),
          val_lev_callback(),
          model_checkpoint_callback
      ]
  )

  model.save_weights(f"model_epoch_{N_EPOCHS}_supply.h5")

# Training on Main dataset

In [ ]:
if USE_VAL:
  # Load the pre-train weight of supplyment set here 
  model.load_weights('/content/model_epoch_80_supply.h5')

  history = model.fit(
      train_dataset,
      validation_data=val_dataset,
      epochs=N_EPOCHS ,
      verbose = 2,
      callbacks=[
          save_model_callback(),
          lr_callback,
          WeightDecayCallback(),
          val_lev_callback(),
          model_checkpoint_callback
      ]
  )

In [ ]:
model.save_weights(f"model_epoch_{N_EPOCHS}.h5")

# Checkpoint for extra epoch and continue training

In [ ]:
# model.load_weights("model_checkpoint.h5")
# checkpoint = ModelCheckpoint('model_checkpoint.h5', monitor='loss', verbose=1, save_best_only=True, mode='min')

# optimizer = tfa.optimizers.RectifiedAdam(sma_threshold=4)
# optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=5)
# model.compile(optimizer=optimizer, loss=CTCLoss)

# history = model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=20,  # Number of extra epochs
#     verbose=2,
#     callbacks=[
#         save_model_callback(),
#         lr_callback,
#         WeightDecayCallback(),
#         val_lev_callback(),
#         checkpoint  # Updated checkpoint callback
#     ]
# )